In [2]:
from urllib.request import urlopen
import certifi
import json
from datetime import datetime

import pandas as pd

In [3]:
API_KEY = "e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"

FETCH_STOCKS_URL = (
    f"https://financialmodelingprep.com/api/v3/stock-screener?apikey={API_KEY}&"
    f"exchange=nyse,nasdaq&isEtf=false&isFund=false&isActivelyTrading=true&limit=10000"
)

# gross profit, earningspersharebasic
FETCH_INCOME_URL = f"https://financialmodelingprep.com/api/v3/income-statement/%s?period=quarter&limit=4&apikey={API_KEY}"
# totalAssets
FETCH_BALANCE_URL = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/%s?period=quarter&limit=4&apikey={API_KEY}"
# priceToSalesRatio
FETCH_RATIO_URL = f"https://financialmodelingprep.com/api/v3/ratios/%s?period=quarter&limit=4&apikey={API_KEY}"
# grossProfitGrowth, ebitgrowth
FETCH_GROWTH_URL = f"https://financialmodelingprep.com/api/v3/financial-growth/%s?period=quarter&limit=4&apikey={API_KEY}"

In [4]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [5]:
stocks = get_jsonparsed_data(FETCH_STOCKS_URL)
len(stocks)

/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_11631/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


5544

In [6]:
tickers = list(map(lambda stock: stock["symbol"], stocks))
tickers = sorted(tickers)

In [7]:
# 0 < psr < 2
# 0 < gross profit / total asset < 2
# 0 < gross_profit
# 0 < eps
# 0.1 < ebit growth
# 0.1 < gross profit growth

In [8]:
def fetch_metric(key, data):
    return data[key] if key in data else 0

def fetch_metrics(ticker):
    incomes = get_jsonparsed_data(FETCH_INCOME_URL % ticker)
    balances = get_jsonparsed_data(FETCH_BALANCE_URL % ticker)
    ratios = get_jsonparsed_data(FETCH_RATIO_URL % ticker)
    growths = get_jsonparsed_data(FETCH_GROWTH_URL % ticker)

    results = []

    size = min(len(incomes), len(balances), len(ratios), len(growths))

    for i in range(size):
        income = incomes[i]
        balance = balances[i]
        ratio = ratios[i]
        growth = growths[i]

        date = datetime.strptime(income["fillingDate"], "%Y-%m-%d")

        results.append(build_key_metrics(date, income, balance, ratio, growth))

    return results


def build_key_metrics(date, income, balance, ratio, growth):
    grossProfit = fetch_metric("grossProfit", income)
    totalAssets = fetch_metric("totalAssets", balance)

    return {
        "date": date,
        "year": date.year,
        "month": date.month,
        "period": income["period"],
        "symbol": income["symbol"],
        "revenue": fetch_metric("revenue", income),
        "gross_profit": grossProfit,
        # 매출 총 이익률 (매출 - 매출 원가) / 매출 -> 영업 효율 척도
        "gross_profit_ratio": fetch_metric("grossProfitRatio", income),
        # earnings before interest, taxes, depreciation and amortization
        # 순이익 + 감가상각비 (유 & 무형) + 세금 + 이자
        # 기업이 건강한지를 보여주는 지표
        "ebitda": fetch_metric("ebitda", income),
        # ev / ebitda
        "ebitda_ratio": fetch_metric("ebitdaratio", income),
        "operating_income": fetch_metric("operatingIncome", income),
        # 순이익
        "net_income": fetch_metric("netIncome", income),
        "net_income_ratio": fetch_metric("netIncomeRatio", income),
        "eps": fetch_metric("eps", income),
        "eps_diluted": fetch_metric("epsdiluted", income),
        "assets": totalAssets,
        "equity": fetch_metric("totalEquity", balance),
        "debt": fetch_metric("totalDebt", balance),
        "net_debt": fetch_metric("netDebt", balance),
        "gp_a": grossProfit / totalAssets if totalAssets != 0 else 0,
        # 유동 비율 : 유동 자산 / 유동 부채. 높을수록 안전
        "current_ratio": fetch_metric("currentRatio", ratio),
        # 당좌 비율 : 당좌 자산 / 유동 부채.
        "quick_ratio": fetch_metric("quickRatio", ratio),
        "cash_ratio": fetch_metric("cashRatio", ratio),
        "operating_profit_margin": fetch_metric("operatingProfitMargin", ratio),
        # return on assets. 총자산수익률
        "roa": fetch_metric("returnOnAssets", ratio),
        # return on equity
        "roe": fetch_metric("returnOnEquity", ratio),
        "net_income_per_ebt": fetch_metric("netIncomePerEBT", ratio),
        "debt_ratio": fetch_metric("debtRatio", ratio),
        "per": fetch_metric("priceEarningsRatio", ratio),
        "pbr": fetch_metric("priceToBookRatio", ratio),
        "psr": fetch_metric("priceToSalesRatio", ratio),
        "pcfr": fetch_metric("priceCashFlowRatio", ratio),
        "pegr": fetch_metric("priceEarningsToGrowthRatio", ratio),
        "revenue_growth": fetch_metric("revenueGrowth", growth),
        "gross_profit_growth": fetch_metric("grossProfitGrowth", growth),
        "ebit_growth": fetch_metric("ebitgrowth", growth),
        "operating_income_growth": fetch_metric("operatingIncomeGrowth", growth),
        "net_income_growth": fetch_metric("netIncomeGrowth", growth),
        "eps_growth": fetch_metric("epsgrowth", growth),
        "eps_diluted_growth": fetch_metric("epsdilutedGrowth", growth),
        "operating_cash_flow_growth": fetch_metric("operatingCashFlowGrowth", growth)
    }

def corps_total_rank_by_standard_columns(df, column_map) -> pd.DataFrame:
    report_df = df.copy()
    rank_df = pd.DataFrame(index=report_df.index)
    total_value = pd.Series([0] * len(report_df), index=report_df.index)

    for col, ascending in column_map.items():
        total_value += report_df[col].rank(ascending=ascending)

    rank_df['total_rank'] = total_value.rank()
    rank_df = rank_df.sort_values(by='total_rank')
    report_df['rank'] = rank_df['total_rank']

    return report_df.loc[rank_df.index]

In [9]:
metrics = []

for ticker in tickers:
    try:
        metrics.append(fetch_metrics(ticker))
        print(f"======= {ticker} ========")
    except:
        None

/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_11631/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


======= A ========
======= AA ========
======= AACG ========
======= AACT ========
======= AADI ========
======= AAGR ========
======= AAL ========
======= AAME ========
======= AAN ========
======= AAOI ========
======= AAON ========
======= AAP ========
======= AAPL ========
======= AAT ========
======= AB ========
======= ABAT ========
======= ABBV ========
======= ABCB ========
======= ABCL ========
======= ABEO ========
======= ABEV ========
======= ABG ========
======= ABL ========
======= ABLLL ========
======= ABLV ========
======= ABM ========
======= ABNB ========
======= ABOS ========
======= ABR ========
======= ABSI ========
======= ABT ========
======= ABTS ========
======= ABUS ========
======= ABVC ========
======= ABVE ========
======= ABVX ========
======= AC ========
======= ACA ========
======= ACAB ========
======= ACAD ========
======= ACB ========
======= ACCD ========
======= ACCO ========
======= ACDC ========
======= ACEL ========
======= ACET ========
=======

======= ARKO ========
======= ARKR ========
======= ARL ========
======= ARLO ========
======= ARLP ========
======= ARM ========
======= ARMK ========
======= AROC ========
======= AROW ========
======= ARQ ========
======= ARQQ ========
======= ARQT ========
======= ARR ========
======= ARRY ========
======= ARTL ========
======= ARTNA ========
======= ARTV ========
======= ARTW ========
======= ARVN ========
======= ARW ========
======= ARWR ========
======= AS ========
======= ASA ========
======= ASAI ========
======= ASAN ========
======= ASB ========
======= ASBA ========
======= ASC ========
======= ASGN ========
======= ASH ========
======= ASIX ========
======= ASLE ========
======= ASMB ========
======= ASML ========
======= ASND ========
======= ASNS ========
======= ASO ========
======= ASPI ========
======= ASPN ========
======= ASPS ========
======= ASPU ========
======= ASR ========
======= ASRT ========
======= ASRV ========
======= ASST ========
======= ASTC ========


======= BOLT ========
======= BON ========
======= BOOM ========
======= BOOT ========
======= BORR ========
======= BOSC ========
======= BOTJ ========
======= BOW ========
======= BOWL ========
======= BOX ========
======= BOXL ========
======= BP ========
======= BPMC ========
======= BPOP ========
======= BPRN ========
======= BPTH ========
======= BPYPM ========
======= BPYPO ========
======= BQ ========
======= BR ========
======= BRAG ========
======= BRBR ========
======= BRC ========
======= BRCC ========
======= BRDG ========
======= BREA ========
======= BRFH ========
======= BRFS ========
======= BRID ========
======= BRK-A ========
======= BRK-B ========
======= BRKH ========
======= BRKL ========
======= BRKR ========
======= BRLS ========
======= BRLT ========
======= BRNS ========
======= BRO ========
======= BROG ========
======= BROS ========
======= BRSP ========
======= BRT ========
======= BRTX ========
======= BRX ========
======= BRY ========
======= BRZE =======

======= CMPX ========
======= CMRA ========
======= CMRE ========
======= CMRX ========
======= CMS ========
======= CMSA ========
======= CMSC ========
======= CMSD ========
======= CMTG ========
======= CMTL ========
======= CNA ========
======= CNC ========
======= CNDT ========
======= CNET ========
======= CNEY ========
======= CNF ========
======= CNFR ========
======= CNGL ========
======= CNH ========
======= CNI ========
======= CNK ========
======= CNM ========
======= CNMD ========
======= CNNE ========
======= CNO ========
======= CNOB ========
======= CNP ========
======= CNQ ========
======= CNS ========
======= CNSL ========
======= CNSP ========
======= CNTA ========
======= CNTB ========
======= CNTM ========
======= CNTX ========
======= CNTY ========
======= CNVS ========
======= CNX ========
======= CNXC ========
======= CNXN ========
======= COCH ========
======= COCO ========
======= COCP ========
======= CODA ========
======= CODI ========
======= CODX ========
=

...

======= PBM ========
======= PBMWW ========
======= PBPB ========
======= PBR ========
======= PBR-A ========
======= PBYI ========
======= PCAR ========
======= PCB ========
======= PCG ========
======= PCH ========
======= PCOR ========
======= PCRX ========
======= PCSA ========
======= PCSC ========
======= PCT ========
======= PCTY ========
======= PCVX ========
======= PCYO ========
======= PD ========
======= PDCC ========
======= PDCO ========
======= PDD ========
======= PDEX ========
======= PDFS ========
======= PDLB ========
======= PDM ========
======= PDS ========
======= PDSB ========
======= PDYN ========
======= PEB ========
======= PEBK ========
======= PEBO ========
======= PECO ========
======= PEG ========
======= PEGA ========
======= PEGY ========
======= PEN ========
======= PENN ========
======= PEP ========
======= PEPG ========
======= PERF ========
======= PERI ========
======= PESI ========
======= PET ========
======= PETQ ========
======= PETS ========
==

======= REVB ========
======= REVG ========
======= REX ========
======= REXR ========
======= REYN ========
======= REZI ========
======= RF ========
======= RF-PF ========
======= RFAI ========
======= RFAIR ========
======= RFIL ========
======= RFL ========
======= RGA ========
======= RGC ========
======= RGCO ========
======= RGEN ========
======= RGF ========
======= RGLD ========
======= RGLS ========
======= RGNX ========
======= RGP ========
======= RGR ========
======= RGS ========
======= RGTI ========
======= RH ========
======= RHI ========
======= RHP ========
======= RIBT ========
======= RICK ========
======= RIG ========
======= RIGL ========
======= RILY ========
======= RILYM ========
======= RIME ========
======= RIO ========
======= RIOT ========
======= RITM ========
======= RITR ========
======= RIVN ========
======= RJF ========
======= RKDA ========
======= RKLB ========
======= RKT ========
======= RL ========
======= RLAY ========
======= RLI ========
======

======= SMTI ========
======= SMWB ========
======= SMX ========
======= SMXT ========
======= SN ========
======= SNA ========
======= SNAL ========
======= SNAP ========
======= SNAX ========
======= SNBR ========
======= SNCR ========
======= SNCY ========
======= SND ========
======= SNDA ========
======= SNDL ========
======= SNDR ========
======= SNDX ========
======= SNES ========
======= SNEX ========
======= SNFCA ========
======= SNGX ========
======= SNN ========
======= SNOA ========
======= SNOW ========
======= SNPS ========
======= SNPX ========
======= SNSE ========
======= SNT ========
======= SNTG ========
======= SNTI ========
======= SNV ========
======= SNX ========
======= SNY ========
======= SO ========
======= SOBR ========
======= SOC ========
======= SOFI ========
======= SOFO ========
======= SOGP ========
======= SOHO ========
======= SOHU ========
======= SOJC ========
======= SOJD ========
======= SOJE ========
======= SOL ========
======= SOLV ========
=

======= TNFA ========
======= TNGX ========
======= TNK ========
======= TNL ========
======= TNON ========
======= TNXP ========
======= TNYA ========
======= TOI ========
======= TOL ========
======= TOMZ ========
======= TOP ========
======= TOPS ========
======= TORO ========
======= TOST ========
======= TOUR ========
======= TOVX ========
======= TOWN ========
======= TOYO ========
======= TPB ========
======= TPC ========
======= TPG ========
======= TPGXL ========
======= TPH ========
======= TPIC ========
======= TPL ========
======= TPR ========
======= TPST ========
======= TPVG ========
======= TPX ========
======= TR ========
======= TRAK ========
======= TRAW ========
======= TRC ========
======= TRDA ========
======= TREE ========
======= TREX ========
======= TRGP ========
======= TRI ========
======= TRIB ========
======= TRIN ========
======= TRINI ========
======= TRINZ ========
======= TRIP ========
======= TRMB ========
======= TRMD ========
======= TRMK ========
=

======= WABC ========
======= WAFD ========
======= WAFU ========
======= WAL ========
======= WALD ========
======= WAT ========
======= WATT ========
======= WAVE ========
======= WAVS ========
======= WAY ========
======= WB ========
======= WBA ========
======= WBD ========
======= WBS ========
======= WBTN ========
======= WBUY ========
======= WBX ========
======= WCC ========
======= WCN ========
======= WD ========
======= WDAY ========
======= WDC ========
======= WDFC ========
======= WDH ========
======= WDS ========
======= WEAV ========
======= WEC ========
======= WEL ========
======= WELL ========
======= WEN ========
======= WERN ========
======= WES ========
======= WEST ========
======= WEX ========
======= WEYS ========
======= WF ========
======= WFC ========
======= WFCF ========
======= WFG ========
======= WFRD ========
======= WGO ========
======= WGS ========
======= WH ========
======= WHD ========
======= WHF ========
======= WHFCL ========
======= WHG ======

In [26]:
result = []

for item in metrics:
    result.extend(item)

In [27]:
stocks_df = pd.DataFrame(sum(metrics, []))
# stocks_df.to_csv('./2024_Q3.csv', index=False)

In [58]:
filtered_stocks = stocks_df[
    (stocks_df['month'].isin([7, 8, 9, 10])) & (stocks_df['year'] == 2024)
].query((
    "0 < psr and psr < 5 and "
    "0 < gp_a and gp_a < 2 and "
    "0 < gross_profit and "
    "0 < eps and "
    "0.1 < ebit_growth and "
    "0.1 < gross_profit_growth"
))
sorted_stocks = corps_total_rank_by_standard_columns(filtered_stocks, {
    'psr': True,
    'gp_a': False
})

In [59]:
sorted_stocks[["date", "year", "month", "period", "symbol", "psr", "gp_a", "gross_profit", "eps", "ebit_growth", "gross_profit_growth"]]

,date,year,month,period,symbol,psr,gp_a,gross_profit,eps,ebit_growth,gross_profit_growth
203,2024-07-23,2024,7,Q1,ACI,0.477131,0.239314,6.240600e+09,0.4200,0.147501,0.114512
20360,2024-09-17,2024,9,Q2,VNCE,0.260985,0.138517,3.513100e+07,0.0500,1.556651,0.174439
10555,2024-09-05,2024,9,Q2,JWN,0.890455,0.172741,1.493000e+09,0.7400,10.047619,0.318905
14408,2024-08-08,2024,8,Q2,OPFI,0.802718,0.147163,8.379100e+07,0.1600,1.420187,0.385227
17713,2024-08-13,2024,8,Q2,SNAL,0.307837,0.107478,8.102476e+06,0.2800,2.916582,3.068135
...,...,...,...,...,...,...,...,...,...,...,...
16699,2024-08-08,2024,8,Q1,RVSB,4.788782,0.011549,1.776600e+07,0.0458,1.368585,0.201623
11099,2024-08-13,2024,8,Q2,LARK,4.916149,0.014032,2.190000e+07,0.5500,0.303668,907.713693
7138,2024-08-12,2024,8,Q2,FCCO,4.934924,0.013520,2.548300e+07,0.4300,0.196663,7.003455
14883,2024-08-08,2024,8,Q2,PDLB,4.970872,0.013909,3.953000e+07,0.1400,0.166442,0.908740


In [36]:
RATING_URL = "https://financialmodelingprep.com/api/v3/rating/%s?apikey=e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"

In [60]:
result = []

for symbol in sorted_stocks['symbol']:
    print(f"fetch {symbol}")
    result.extend(get_jsonparsed_data(RATING_URL % symbol))

fetch ACI


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_11631/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


fetch VNCE
fetch JWN
fetch OPFI
fetch SNAL
fetch COMP
fetch TKLF
fetch DXYN
fetch CWH
fetch EXPI
fetch SDHC
fetch PFGC
fetch DTIL
fetch LTRPA
fetch PRPL
fetch TIPT
fetch LESL
fetch CBRL
fetch KSS
fetch MSS
fetch EQ
fetch CLOV
fetch JAKK
fetch CRMT
fetch VRA
fetch RAIL
fetch USFD
fetch ADAP
fetch HZO
fetch DXPE
fetch VATE
fetch GOSS
fetch NISN
fetch ESOA
fetch HBB
fetch MG
fetch ONEW
fetch PAL
fetch CAPL
fetch SCS
fetch SCHL
fetch KPTI
fetch TG
fetch FNKO
fetch TREE
fetch IBEX
fetch GO
fetch BWNB
fetch BW
fetch BWSN
fetch ARAY
fetch VEON
fetch GAP
fetch COMM
fetch LX
fetch VIRC
fetch FSTR
fetch ASO
fetch ANDE
fetch AAL
fetch SMP
fetch PETQ
fetch ARC
fetch POLA
fetch CNXN
fetch RMTI
fetch MRX
fetch NMRK
fetch SRI
fetch GTIM
fetch FORR
fetch PPIH
fetch STRT
fetch CVS
fetch DSWL
fetch TILE
fetch LTRX
fetch TWIN
fetch SHYF
fetch MLR
fetch NWL
fetch GTEC
fetch PBPB
fetch XRX
fetch GLP
fetch GIC
fetch VHI
fetch BHC
fetch ASIX
fetch WNC
fetch SIF
fetch PLOW
fetch GT
fetch MYE
fetch LCII
fetch 

In [61]:
rating_df = pd.DataFrame(result)
# rating_df.query("ratingScore >= 4")
rating_df

,symbol,date,rating,ratingScore,ratingRecommendation,ratingDetailsDCFScore,ratingDetailsDCFRecommendation,ratingDetailsROEScore,ratingDetailsROERecommendation,ratingDetailsROAScore,ratingDetailsROARecommendation,ratingDetailsDEScore,ratingDetailsDERecommendation,ratingDetailsPEScore,ratingDetailsPERecommendation,ratingDetailsPBScore,ratingDetailsPBRecommendation
0,ACI,2024-10-04,B+,3,Neutral,5,Strong Buy,5,Strong Buy,3,Neutral,1,Strong Sell,3,Neutral,2,Sell
1,VNCE,2024-10-04,B+,3,Neutral,5,Strong Buy,2,Sell,2,Sell,1,Strong Sell,3,Neutral,5,Strong Buy
2,JWN,2024-10-04,B-,3,Neutral,1,Strong Sell,5,Strong Buy,4,Buy,1,Strong Sell,3,Neutral,1,Strong Sell
3,OPFI,2024-10-04,B-,3,Neutral,1,Strong Sell,5,Strong Buy,3,Neutral,1,Strong Sell,3,Neutral,2,Sell
4,SNAL,2024-10-04,A-,4,Buy,1,Strong Sell,5,Strong Buy,5,Strong Buy,2,Sell,4,Buy,4,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,RVSB,2024-10-04,B-,3,Neutral,4,Buy,2,Sell,2,Sell,2,Sell,1,Strong Sell,4,Buy
333,LARK,2024-10-04,B+,3,Neutral,5,Strong Buy,3,Neutral,3,Neutral,2,Sell,3,Neutral,3,Neutral
334,FCCO,2024-10-04,B+,3,Neutral,5,Strong Buy,3,Neutral,3,Neutral,2,Sell,3,Neutral,3,Neutral
335,PDLB,2024-10-04,B,3,Neutral,4,Buy,2,Sell,3,Neutral,2,Sell,2,Sell,4,Buy


In [48]:
GRADE_URL = "https://financialmodelingprep.com/api/v3/grade/%s?apikey=e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c&limit=1"

In [62]:
result = []

for symbol in rating_df.query("ratingScore >= 4")['symbol']:
    print(f"fetch {symbol}")
    result.extend(get_jsonparsed_data(GRADE_URL % symbol))

fetch SNAL


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_11631/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


fetch TKLF
fetch EXPI
fetch MSS
fetch JAKK
fetch VRA
fetch HZO
fetch NISN
fetch ESOA
fetch SCS
fetch TG
fetch IBEX
fetch LX
fetch VIRC
fetch ASO
fetch ANDE
fetch SMP
fetch ARC
fetch POLA
fetch CNXN
fetch GTIM
fetch FORR
fetch STRT
fetch DSWL
fetch TWIN
fetch MLR
fetch GTEC
fetch GIC
fetch VHI
fetch ASIX
fetch WNC
fetch PLOW
fetch SPLP
fetch CVEO
fetch MT
fetch WKC
fetch IESC
fetch AP
fetch CENT
fetch DFH
fetch TEX
fetch URBN
fetch HOUS
fetch LWAY
fetch BLDR
fetch HG
fetch SITE
fetch RGS
fetch BBSI
fetch UHG
fetch SOHO
fetch FDP
fetch ALK
fetch PSO
fetch PNRG
fetch SWIM
fetch ETD
fetch CUK
fetch CCL
fetch XYF
fetch UAL
fetch DAL
fetch SPNT
fetch HLLY
fetch EPC
fetch SLGL
fetch OSK
fetch FULC
fetch UFPI
fetch APOG
fetch PFIE
fetch MPC
fetch CBAT
fetch LBRT
fetch CALM
fetch TPH
fetch ARCB
fetch NOV
fetch MTH
fetch VC
fetch GRBK
fetch EME
fetch FLR
fetch TMHC
fetch NWPX
fetch LAUR
fetch HURN
fetch IMMR
fetch DHI
fetch IP
fetch PHM
fetch SAN
fetch TAP
fetch BTI
fetch EE
fetch AMPY
fetch HII

In [63]:
grade_df = pd.DataFrame(result)
grade_df

,symbol,date,gradingCompany,previousGrade,newGrade
0,SNAL,2023-04-13,Noble Capital Markets,Outperform,Outperform
1,EXPI,2024-08-01,DA Davidson,Neutral,Neutral
2,JAKK,2024-08-01,Small Cap Consumer Research,Buy,Buy
3,VRA,2024-09-16,Small Cap Consumer Research,Buy,Buy
4,HZO,2024-09-09,Citigroup,Neutral,Buy
...,...,...,...,...,...
97,PK,2024-09-26,Wolfe Research,Outperform,Peer Perform
98,CLDT,2024-04-17,Barclays,Overweight,Overweight
99,CBNK,2024-07-23,"Keefe, Bruyette & Woods",Market Perform,Market Perform
100,RUN,2024-09-25,Goldman Sachs,Buy,Buy


In [96]:
grade_df.set_index('symbol')[['previousGrade', 'newGrade']].join(rating_df.set_index('symbol')[['rating', 'ratingScore']]).loc[grade_df['symbol']].head(20)

,previousGrade,newGrade,rating,ratingScore
symbol,,,,
SNAL,Outperform,Outperform,A-,4
EXPI,Neutral,Neutral,A-,4
JAKK,Buy,Buy,A-,4
VRA,Buy,Buy,A-,4
HZO,Neutral,Buy,A-,4
SCS,Buy,Buy,A,4
TG,Neutral,Neutral,A,4
IBEX,Outperform,Outperform,A,4
LX,Buy,Neutral,A+,4
